In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv("raw_datasets/chanlamcoupling_all_flat_tidied.csv")
df

,reaction_id,outcomes,inputs_Boronic Acid_components,inputs_Sulfonamide_components,inputs_Catalyst_components,Inputs_Base
0,ord-8757937df9d044d585ca73bbef0d97e7,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ..."
1,ord-58dad3b796614838bef21ce3ca060176,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ..."
2,ord-47b42e853d9f4360a4d36bb453c1c3b3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ..."
3,ord-a54883a3bc3148bab8f981197d4679e3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ..."
4,ord-10dd922af989439a935a71322d9359b0,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ..."
...,...,...,...,...,...,...
9627,ord-800daa953ad14ca4951362205acfda11,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",NaN
9628,ord-0a6e8091bd8d451ab13d4169787587db,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",NaN
9629,ord-a84fa7ed1b55455d9d4f4c4f9a5e5849,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",NaN
9630,ord-9cbf9dc7f17c48d6804394ff4ba09725,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",NaN


In [3]:
import pprint
import ast
import pandas as pd


In [4]:
temp_json = df["inputs_Boronic Acid_components"]
print(temp_json)

solvent_smiles_list = []
boronic_acid_smiles_list = []
boronic_acid_ratio_list = []

for i in range(0, len(temp_json)):
    # Parse safely into a Python list of dicts
    try:
        data = ast.literal_eval(temp_json[i])
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing JSON data at index {i}: {e}")
        break

    # Flatten relevant fields
    flattened = []
    for item in data:
        smiles = None
        for i in item.get("identifiers", []):
            if i.get("type") == "SMILES":
                smiles = i.get("value")
                break

        amount_type = list(item.get("amount", {}).keys())[0] if item.get("amount") else None
        amount_value = item.get("amount", {}).get(amount_type, {}).get("value") if amount_type else None
        amount_units = item.get("amount", {}).get(amount_type, {}).get("units") if amount_type else None

        flattened.append({
            "smiles": smiles,
            "reaction_role": item.get("reaction_role"),
            "amount_type": amount_type,
            "amount_value": amount_value,
            "amount_units": amount_units
        })
        
    #pprint.pprint(flattened)

    solvent_smiles = flattened[1]["smiles"]
    boronic_acid_smiles = flattened[0]["smiles"]
    boronic_acid_ratio = flattened[0]["amount_value"]
    #print(solvent_smiles, boronic_acid_smiles, boronic_acid_ratio)
    solvent_smiles_list.append(solvent_smiles)
    boronic_acid_smiles_list.append(boronic_acid_smiles)
    boronic_acid_ratio_list.append(boronic_acid_ratio)
    
#print(solvent_smiles_list)
df["solvent_smiles"] = solvent_smiles_list
df["boronic_acid_smiles"] = boronic_acid_smiles_list
df["boronic_acid_ratio"] = boronic_acid_ratio_list

df.head(5)


0       [{'identifiers': [{'type': 'SMILES', 'value': ...
1       [{'identifiers': [{'type': 'SMILES', 'value': ...
2       [{'identifiers': [{'type': 'SMILES', 'value': ...
3       [{'identifiers': [{'type': 'SMILES', 'value': ...
4       [{'identifiers': [{'type': 'SMILES', 'value': ...
                              ...                        
9627    [{'identifiers': [{'type': 'SMILES', 'value': ...
9628    [{'identifiers': [{'type': 'SMILES', 'value': ...
9629    [{'identifiers': [{'type': 'SMILES', 'value': ...
9630    [{'identifiers': [{'type': 'SMILES', 'value': ...
9631    [{'identifiers': [{'type': 'SMILES', 'value': ...
Name: inputs_Boronic Acid_components, Length: 9632, dtype: object


,reaction_id,outcomes,inputs_Boronic Acid_components,inputs_Sulfonamide_components,inputs_Catalyst_components,Inputs_Base,solvent_smiles,boronic_acid_smiles,boronic_acid_ratio
0,ord-8757937df9d044d585ca73bbef0d97e7,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0
1,ord-58dad3b796614838bef21ce3ca060176,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0
2,ord-47b42e853d9f4360a4d36bb453c1c3b3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0
3,ord-a54883a3bc3148bab8f981197d4679e3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0
4,ord-10dd922af989439a935a71322d9359b0,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0


In [6]:
temp_json = df["inputs_Sulfonamide_components"]
print(temp_json[0])

solvent_smiles_list = []
sulfonamide_smiles_list = []
sulfonamide_ratio_list = []

for i in range(0, len(temp_json)):
    # Parse safely into a Python list of dicts
    try:
        data = ast.literal_eval(temp_json[i])
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing JSON data at index {i}: {e}")
        break

    # Flatten relevant fields
    flattened = []
    for item in data:
        smiles = None
        for i in item.get("identifiers", []):
            if i.get("type") == "SMILES":
                smiles = i.get("value")
                break

        amount_type = list(item.get("amount", {}).keys())[0] if item.get("amount") else None
        amount_value = item.get("amount", {}).get(amount_type, {}).get("value") if amount_type else None
        amount_units = item.get("amount", {}).get(amount_type, {}).get("units") if amount_type else None

        flattened.append({
            "smiles": smiles,
            "reaction_role": item.get("reaction_role"),
            "amount_type": amount_type,
            "amount_value": amount_value,
            "amount_units": amount_units
        })
        
    #pprint.pprint(flattened)

    solvent_smiles = flattened[1]["smiles"]
    sulfonamide_smiles = flattened[0]["smiles"]
    sulfonamide_ratio = flattened[0]["amount_value"]
    #print(solvent_smiles, boronic_acid_smiles, boronic_acid_ratio)
    solvent_smiles_list.append(solvent_smiles)
    sulfonamide_smiles_list.append(sulfonamide_smiles)
    sulfonamide_ratio_list.append(sulfonamide_ratio)
    
#print(solvent_smiles_list)
df["solvent_smiles"] = solvent_smiles_list
df["sulfonamide_smiles"] = sulfonamide_smiles_list
df["sulfonamide_ratio"] = sulfonamide_ratio_list

df.head(5)


[{'identifiers': [{'type': 'SMILES', 'value': 'BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1'}], 'amount': {'moles': {'value': 10.0, 'units': 'MICROMOLE'}}, 'reaction_role': 'REACTANT'}, {'identifiers': [{'type': 'SMILES', 'value': 'ClCCCl'}], 'amount': {'volume': {'value': 100.0, 'units': 'MICROLITER'}, 'volume_includes_solutes': True}, 'reaction_role': 'SOLVENT'}]


,reaction_id,outcomes,inputs_Boronic Acid_components,inputs_Sulfonamide_components,inputs_Catalyst_components,Inputs_Base,solvent_smiles,boronic_acid_smiles,boronic_acid_ratio,sulfonamide_smiles,sulfonamide_ratio
0,ord-8757937df9d044d585ca73bbef0d97e7,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0
1,ord-58dad3b796614838bef21ce3ca060176,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0
2,ord-47b42e853d9f4360a4d36bb453c1c3b3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0
3,ord-a54883a3bc3148bab8f981197d4679e3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0
4,ord-10dd922af989439a935a71322d9359b0,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=CC(CS(=O)(N)=O)=NO1,10.0


In [14]:
temp_json = df["Inputs_Base"]
print(temp_json[0])

base_smiles_list = []
base_smiles_list = []
base_ratio_list = []

for i in range(0, len(temp_json)):
    # Parse safely into a Python list of dicts
    try:
        data = ast.literal_eval(temp_json[i])
    except (ValueError, SyntaxError) as e:
        # print(f"Error parsing JSON data at index {i}: {e}")
        base_smiles = ""
        base_ratio = 0
    else:
        # Flatten relevant fields
        flattened = []
        for item in data:
            smiles = None
            for i in item.get("identifiers", []):
                if i.get("type") == "SMILES":
                    smiles = i.get("value")
                    break

            amount_type = list(item.get("amount", {}).keys())[0] if item.get("amount") else None
            amount_value = item.get("amount", {}).get(amount_type, {}).get("value") if amount_type else None
            amount_units = item.get("amount", {}).get(amount_type, {}).get("units") if amount_type else None

            flattened.append({
                "smiles": smiles,
                "reaction_role": item.get("reaction_role"),
                "amount_type": amount_type,
                "amount_value": amount_value,
                "amount_units": amount_units
            })
            
        #pprint.pprint(flattened)

        base_smiles = flattened[0]["smiles"]
        base_ratio = flattened[0]["amount_value"]
        #print(solvent_smiles, boronic_acid_smiles, boronic_acid_ratio)
    finally:
        base_smiles_list.append(base_smiles)
        base_ratio_list.append(base_ratio)
    
# #print(solvent_smiles_list)
df["base_smiles"] = base_smiles_list
df["base_ratio"] = base_ratio_list
print(base_smiles_list)

df.head(5)


[{'identifiers': [{'type': 'SMILES', 'value': '[F-].[Cs+]'}], 'amount': {'moles': {'value': 20.0, 'units': 'MICROMOLE'}}, 'reaction_role': 'REAGENT'}]
['[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[F-].[Cs+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O.[K+].[K+].[K+]', '[O-]P([O-])([O-])=O

,reaction_id,outcomes,inputs_Boronic Acid_components,inputs_Sulfonamide_components,inputs_Catalyst_components,Inputs_Base,solvent_smiles,boronic_acid_smiles,boronic_acid_ratio,sulfonamide_smiles,sulfonamide_ratio,base_smiles,base_ratio
0,ord-8757937df9d044d585ca73bbef0d97e7,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0,[F-].[Cs+],20.0
1,ord-58dad3b796614838bef21ce3ca060176,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0,[F-].[Cs+],20.0
2,ord-47b42e853d9f4360a4d36bb453c1c3b3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0,[F-].[Cs+],20.0
3,ord-a54883a3bc3148bab8f981197d4679e3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0,[F-].[Cs+],20.0
4,ord-10dd922af989439a935a71322d9359b0,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=CC(CS(=O)(N)=O)=NO1,10.0,[F-].[Cs+],20.0


In [16]:
temp_json = df["inputs_Catalyst_components"]
print(temp_json[0])

catalyst_smiles_list = []
catalyst_ratio_list = []

for i in range(0, len(temp_json)):
    # Parse safely into a Python list of dicts
    try:
        data = ast.literal_eval(temp_json[i])
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing JSON data at index {i}: {e}")
        break

    # Flatten relevant fields
    flattened = []
    for item in data:
        smiles = None
        for i in item.get("identifiers", []):
            if i.get("type") == "SMILES":
                smiles = i.get("value")
                break

        amount_type = list(item.get("amount", {}).keys())[0] if item.get("amount") else None
        amount_value = item.get("amount", {}).get(amount_type, {}).get("value") if amount_type else None
        amount_units = item.get("amount", {}).get(amount_type, {}).get("units") if amount_type else None

        flattened.append({
            "smiles": smiles,
            "reaction_role": item.get("reaction_role"),
            "amount_type": amount_type,
            "amount_value": amount_value,
            "amount_units": amount_units
        })
        
    #pprint.pprint(flattened)

    catalyst_smiles = flattened[0]["smiles"]
    catalyst_ratio = flattened[0]["amount_value"]
    #print(solvent_smiles, boronic_acid_smiles, boronic_acid_ratio)
    catalyst_smiles_list.append(catalyst_smiles)
    catalyst_ratio_list.append(catalyst_ratio)
    
#print(solvent_smiles_list)
df["catalyst_smiles"] = catalyst_smiles_list
df["catalyst_ratio"] = catalyst_ratio_list

df.head(5)

[{'identifiers': [{'type': 'SMILES', 'value': 'CC(=O)[O-].CC(=O)[O-].[Cu+2]'}], 'amount': {'moles': {'value': 2.0, 'units': 'MICROMOLE'}}, 'reaction_role': 'CATALYST', 'source': {'vendor': 'Sigma-Aldrich'}}]


,reaction_id,outcomes,inputs_Boronic Acid_components,inputs_Sulfonamide_components,inputs_Catalyst_components,Inputs_Base,solvent_smiles,boronic_acid_smiles,boronic_acid_ratio,sulfonamide_smiles,sulfonamide_ratio,base_smiles,base_ratio,catalyst_smiles,catalyst_ratio
0,ord-8757937df9d044d585ca73bbef0d97e7,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0,[F-].[Cs+],20.0,CC(=O)[O-].CC(=O)[O-].[Cu+2],2.0
1,ord-58dad3b796614838bef21ce3ca060176,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,BrC1=CC(S(=O)(N)=O)=CC(C(OC)=O)=C1,10.0,[F-].[Cs+],20.0,CC(=O)[O-].CC(=O)[O-].[Cu+2],2.0
2,ord-47b42e853d9f4360a4d36bb453c1c3b3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0,[F-].[Cs+],20.0,CC(=O)[O-].CC(=O)[O-].[Cu+2],2.0
3,ord-a54883a3bc3148bab8f981197d4679e3,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(OC)C=C1,15.0,CC1=C(OC(S(=O)(N)=O)=C1)C(OC)=O,10.0,[F-].[Cs+],20.0,CC(=O)[O-].CC(=O)[O-].[Cu+2],2.0
4,ord-10dd922af989439a935a71322d9359b0,"[{'reaction_time': {'value': 18.0, 'units': 'H...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...","[{'identifiers': [{'type': 'SMILES', 'value': ...",ClCCCl,OB(O)C1=CC=C(C(F)(F)F)C=C1,15.0,CC1=CC(CS(=O)(N)=O)=NO1,10.0,[F-].[Cs+],20.0,CC(=O)[O-].CC(=O)[O-].[Cu+2],2.0


In [18]:
df.drop(columns=["inputs_Boronic Acid_components", "inputs_Sulfonamide_components", "Inputs_Base", "inputs_Catalyst_components"], inplace=True)
df.to_csv("raw_datasets/chanlamcoupling_all_flat_tidied_preprocessexcludingoutcomes.csv", index=False)